In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa as tsa
import statsmodels as sm
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
data = pd.read_excel("CONSUMO-2024-05.xlsx", index_col="Fecha", parse_dates=["Fecha"], date_format="%Y-%m-%d ", skiprows=6, skipfooter=3)

consume_data = data[['Gasolina regular', 'Gasolina superior', 'Gas licuado de petróleo']]
consume_data 

In [ ]:
start_date = '2017-12-01'

consume_data['Diesel'] = np.where((data.index <= start_date),
                                    data['Diesel alto azufre'],
                                    data['Diesel bajo azufre'])

consume_data['Diesel']

In [ ]:
inicio = min(consume_data.index)
inicio
values = consume_data.loc[inicio]
print("Fecha: ", inicio, "Valor: ", values['Gasolina superior'])

In [ ]:
fin = max(consume_data.index)
fin
values = consume_data.loc[fin]
print("Fecha: ", fin, "Valor: ", values['Gasolina superior'])

In [ ]:
infer_frec = pd.infer_freq(consume_data['Gasolina superior'].index)
infer_frec

In [ ]:
# Replace zero values with NaN
consume_data = consume_data.replace(0, np.nan)

# Drop rows with NaN values (which were originally zero values)
consume_data = consume_data.dropna()

In [ ]:
import matplotlib.dates as mdates

# Assuming consume_data is already loaded

# Ensure the index is a DatetimeIndex
if not isinstance(consume_data.index, pd.DatetimeIndex):
    consume_data.index = pd.to_datetime(consume_data.index)

# Group the data by year and sum the values
consume_data_yearly = consume_data.resample('Y').sum()

# Plot the aggregated data
plt.plot(consume_data_yearly)

# Set the title and labels
plt.gca().set(title="Cantidad de pasajeros por año", xlabel="Año", ylabel="Cantidad de Pasajeros")

# Format the x-axis to display only the year
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add a legend
plt.legend(consume_data.columns)

In [ ]:
plt.plot(consume_data)
plt.gca().set(title="Consumo de gasolina a través del tiempo", xlabel="Año", ylabel="Consumo")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.legend(consume_data.columns)
plt.show()

In [ ]:
# Se calcula la media móvil y la desviación estandar móvil de los últimos 12 meses.
mediaMovil = consume_data['Gasolina superior'].rolling(window=12).mean()
deMovil = consume_data['Gasolina superior'].rolling(window=12).std()
# Se grafican los resultados.
original = plt.plot(consume_data['Gasolina superior'], color="blue", label="Original")
media = plt.plot(mediaMovil, color='red', label = 'Media Movil')
ds = plt.plot(deMovil,color='black', label = 'Desviación Estándar Móvil')
plt.legend(loc = 'best')
plt.title('Media y desviación estándar móvil de Gasolina Superior')
plt.show(block=False)

In [ ]:
descomposicion = seasonal_decompose(consume_data['Gasolina superior'], model='additive', period=12)
descomposicion.plot()

In [ ]:
print('Resultados del Test de Dickey Fuller')
dfTest = adfuller(consume_data['Gasolina superior'], autolag='AIC')
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
print(salidaDf)

In [ ]:
consume_data = consume_data['Gasolina superior']

train_consume_data = consume_data[:'2021']
test_consume_data = consume_data['2021':]

consume_data_log = np.log(train_consume_data)

In [ ]:
consume_data_log.plot()

In [ ]:
print('Resultados del Test de Dickey Fuller')
consume_data_diff = train_consume_data.diff()
consume_data_diff.dropna(inplace=True)
dfTest = adfuller(consume_data_diff, autolag='AIC')
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
print(salidaDf)

In [ ]:
consume_data_diff.plot()

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

plt.subplot(121)
plt.plot(pacf(consume_data_diff, nlags=36))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación Parcial 36 retardos')

plt.subplot(122)
plt.plot(pacf(consume_data_diff, nlags=5))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación Parcial 5 retardos')

In [ ]:

plt.rcParams['figure.figsize'] = [15, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

#Plot ACF: 
plt.subplot(121) 
plt.plot(acf(consume_data_diff,nlags=12,fft=False))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación con 12 retardos')

plt.subplot(122) 
plt.plot(acf(consume_data_diff,nlags=5,fft=False))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación con 5 retardos')

plt.tight_layout()

In [ ]:
consume_data_log_D = consume_data_log.diff(12)
consume_data_log_D.dropna(inplace=True)

In [ ]:
plt.plot(acf(consume_data_diff,nlags=36,fft=False))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación con 36 retardos')
plt.rcParams['figure.figsize'] = [15, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [ ]:

plt.plot(pacf(consume_data_log_D, nlags=15))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(consume_data_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación Parcial 5 retardos')


P = 2
D = 1
Q = 0

In [ ]:
modelo111 = SARIMAX(consume_data, order=(1,1,1), seasonal_order=(1,1,0,12), enforce_stationarity=False, enforce_invertibility=False)
resultado_m111 = modelo111.fit()
print(resultado_m111.summary().tables[1])

In [ ]:
modelo211 = SARIMAX(consume_data, order=(2,1,1), seasonal_order=(1,1,0,12), enforce_stationarity=False, enforce_invertibility=False)
resultado_m211 = modelo211.fit()
print(resultado_m211.summary().tables[1])

In [ ]:
modelo112 = SARIMAX(consume_data, order=(1,1,2), seasonal_order=(2,1,0,12), enforce_stationarity=False, enforce_invertibility=False)

resultado_m112 = modelo112.fit()
print(resultado_m112.summary().tables[1])

In [ ]:
print("Resultados de AIC (Akaike information criterion)")
print("Modelo 111=",resultado_m111.aic)
print("Modelo 211=",resultado_m211.aic)
print("Modelo 112=",resultado_m112.aic)
print("Resultados de BIC (Bayesian information criterion)")
print("Modelo 111=",resultado_m111.bic)
print("Modelo 211=",resultado_m211.bic)
print("Modelo 112=",resultado_m112.aic)

In [ ]:
pred = resultado_m112.get_prediction(start=test_consume_data.index[0], dynamic=False)
pred_ci = pred.conf_int()
ax = consume_data['2020':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 4))
ax.fill_between(pred_ci.iloc[:,0],
                pred_ci.iloc[:,1], color='k', alpha=.2)
#ax.set_xlabel('Date')
#ax.set_ylabel('Retail_sold')
plt.legend()
plt.show()

pred